renaming 제외, 얼굴 detection과 마진 포함 cropping, facial landmark좌표 찍기

1. 얼굴 Detection하고 원본 이미지 width와 height, face bounding box 좌표 저장

In [2]:
import os
import cv2
import pandas as pd
import tensorflow as tf
from retinaface import RetinaFace

In [2]:
retinaface = RetinaFace()

model[normal quality] init ..
model success !


In [13]:
#from image_cropping_retina.ipynb
def facial_detect_bbox(directory):
    bbox_data = []
    cnt = 1

    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            real_h, real_w = img.shape[:2]

            faces = retinaface.predict(img)
            max_width = 0
            selected_face = None
            for face in faces:
                x1, y1, x2, y2 = face['x1'], face['y1'], face['x2'], face['y2']
                width = x2 - x1
                if width > max_width:
                    max_width = width
                    selected_face = face

            if selected_face is not None:
                if (cnt%100) == 0:
                    print(cnt,'/60000')
                cnt += 1
                x1, y1, x2, y2 = selected_face['x1'], selected_face['y1'], selected_face['x2'], selected_face['y2']
                bbox_data.append([filename, x1, y1, x2, y2, real_w, real_h])
    
    df = pd.DataFrame(bbox_data, columns=['Filename', 'x_Top','y_Top','x_Bottom','y_Bottom', 'real_width', 'real_height'])
    return df

출력 - dataframe반환 파일 파일명 | x_Top | y_Top | x_Bottom | y_Bottom | real_width | real_height

2. 정사각형으로 만들어 주기 위한 4방향 margin 값 조정

In [18]:
def make_square_with_margins(df):
    new_bbox = []
    for index, row in df.iterrows():
        filename = row["Filename"]
        x1 = row["x_Top"]
        y1 = row["y_Top"]
        x2 = row["x_Bottom"]
        y2 = row["y_Bottom"]
        real_w = row["real_width"]
        real_h = row["real_height"]

        w = x2 - x1
        h = y2 - y1
        if h >= w: #세로가 가로보다 긴 경우(길쭉한 경우) -- 양옆으로 마진을 더 줘야한다.
            move = (h-w)/2
            margin_left = move
            margin_right = move
            margin_top = 0
            margin_bottom = 0
        else: #가로가 세로보다 긴 경우(넓적한 경우) -- 위아래로 마진을 더 줘야한다.
            move = (w-h)/2
            margin_left = 0
            margin_right = 0
            margin_top = move
            margin_bottom = move
        w = w + margin_left + margin_right
        h = h + margin_top + margin_bottom
        new_bbox.append([filename,x1,y1,x2,y2,real_w,real_h,
                         margin_left,margin_right,margin_top,margin_bottom,w,h])
    df1 = pd.DataFrame(new_bbox, columns=['Filename','x_Top','y_Top','x_Bottom','y_Bottom','real_width','real_height',
                                          'margin_left','margin_right','margin_top','margin_bottom','width', 'height'])
    return df1

In [19]:
bbox_0 = pd.read_csv("D:/Dataset/aihub_korean_emotion/csvs/detection-bbox/retina-face-detection-bbox-_____.csv", index_col=0)
square_0 = make_square_with_margins(bbox_0)
square_0.to_csv('made-square-_____.csv')

출력 - dataframe반환 파일명 | x_Top | y_Top | x_Bottom | y_Bottom | real_width | real_height |
                    margin_left | margin_right | margin_top | margin_bottom | width | height 
                    --width, height는 같아진 상태이다. 좌표는 이미지 범위를 고려 X

3. 추가 마진 값 - 마진을 넘겨주는 값만큼 4방향 모두 추가해줌 

In [20]:
def extra_margin_extend(df, margin):
    df["margin_left"] = df["margin_left"] + margin
    df["margin_right"] = df["margin_right"] + margin
    df["margin_top"] = df["margin_top"] + margin
    df["margin_bottom"] = df["margin_bottom"] + margin
    df["width"] = df["width"] + margin*2
    df["height"] = df["height"] + margin*2
    return df

In [23]:
df0 = pd.read_csv("D:/Dataset/aihub_korean_emotion/csvs/made-square/made-square-_____.csv", index_col=0)
ex0 = extra_margin_extend(df0, 300)
ex0.to_csv('extramargin-300-_____.csv')

출력 : 2번 단계의 결과물에서 4방향 마진 값만 증가된 상태의 dataframe. 좌표는 이미지 범위를 고려 X. width == height

4. 마진 값 조정 - 640x640보다 사이즈가 작은지, 좌표가 이미지 범위를 벗어나는지 확인하고 조정 
4방향 마진 값만 변화할 수 있으며, 이후 width와 height값은 다를 수도 있음

In [24]:
def ready_to_crop(df):
    new_crop_box = []
    for index, row in df.iterrows():
        filename = row["Filename"]
        x1 = row["x_Top"]
        y1 = row["y_Top"]
        x2 = row["x_Bottom"]
        y2 = row["y_Bottom"]
        real_w = row["real_width"]
        real_h = row["real_height"]
        margin_l = row["margin_left"]
        margin_r = row["margin_right"]
        margin_t = row["margin_top"]
        margin_b = row["margin_bottom"]
        width = row["width"]
        height = row["height"]

        if width < 640:
            move = (640 - width)/2
            margin_l += move
            margin_r += move
            margin_t += move
            margin_b += move
            width = (x2-x1) + margin_l + margin_r
            height = (y2-y1) + margin_t + margin_b
        #경계면 조정
        if (x1-margin_l) < 0:
            margin_l = x1
        if (x2+margin_r) > real_w:
            margin_r = (real_w-x2)
        if (y1-margin_t) < 0:
            margin_t = y1
        if (y2+margin_b) > real_h:
            margin_b = (real_h-y2)
        width = (x2-x1) + margin_l + margin_r
        height = (y2-y1) + margin_t + margin_b
        new_crop_box.append([filename, x1, y1, x2, y2, real_w, real_h,
                             margin_l, margin_r, margin_t, margin_b, width, height])
    df1 = pd.DataFrame(new_crop_box, columns=['Filename','x_Top','y_Top','x_Bottom','y_Bottom','original_width','original_height',
                                              'margin_left','margin_right','margin_top','margin_bottom','width','height'])
    return df1

In [25]:
df0 = pd.read_csv("D:/Dataset/aihub_korean_emotion/csvs/extramargin-300/extramargin-300-_____.csv", index_col=0)
ex0 = ready_to_crop(df0)
ex0.to_csv('dealt-boundary-_____.csv')

출력 - 마진 값이 조정된 dataframe
파일명 | x_Top | y_Top | x_Bottom | y_Bottom | original_width | original_height
| margin_left | margin_right | margin_top | margin_bottom | width | height
이때 width 와 height는 다를 수도 있음!

5. 최종 640 by 640 이미지 자르기.
자르고 resize(축소), interpolation = cv2.INTER_CUBIC

In [4]:
def crop_images_four_margin(directory, save_directory, df):
    for index, row in df.iterrows():
        filename = row["Filename"]
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            x1 = int(row["x_Top"]-row["margin_left"])
            y1 = int(row["y_Top"]-row["margin_top"])
            x2 = int(row["x_Bottom"]+row["margin_right"])
            y2 = int(row["y_Bottom"]+row["margin_bottom"])
            try:
                face_crop = img[y1:y2, x1:x2]
                resized_face = cv2.resize(face_crop, (640, 640), interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(os.path.join(save_directory, filename), resized_face)
                if (index%100) == 0:
                    print(index,'/60000')
            except Exception as e:
                print(str(e))
                continue

In [ ]:
directory = r"D:\Dataset\aihub_korean_emotion\_____"
save_directory = r"D:\Dataset\aihub_korean_emotion\cropped\_____"
df = pd.read_csv("D:/Dataset/aihub_korean_emotion/csvs/dealt-boundary-problem/dealt-boundary-_____.csv", index_col=0)
crop_images_four_margin(directory, save_directory, df)

출력 - 참조디렉터리에 있는 파일들을 열어서 데이터 프레임에 있는 정보대로 이미지를 자르고 리사이징 후 저장디렉터리에 이미지를 write

6. 랜드마크 detection (Dlib) 커널을 myenv로 변경하기.
detection을 다시 하지 않고 4방향 마진값이 resizing 전과 비례해 640x640 상에서 얼마나 차지하는 지를 계산하여 얼굴 범위를 계산하여 넣어줌

In [8]:
import dlib
import numpy as np
import cv2
from imutils import face_utils
import os
import pandas as pd

In [9]:
#모델 준비
p = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(p)

In [10]:
def draw_and_save_landmarks_ratio(directory, save_directory, df):
    mrk_data = []
    cnt = 1
    for filename in os.listdir(directory):
        #print(cnt,'/5000')
        cnt += 1
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            row = df[df["Filename"]==filename]
            row = row.iloc[0]
            ml = row["margin_left"]
            mt = row["margin_top"]
            mr = row["margin_right"]
            mb = row["margin_bottom"]
            width = row["width"]
            height = row["height"]
            X1 = int((640*ml/width))
            Y1 = int((640*mt/height))
            X2 = int(640-(640*mr/width))
            Y2 = int(640-(640*mb/height))

            rect = dlib.rectangle(X1,Y1,X2,Y2)

            shape = predictor(gray, rect)
            if shape is not None:
                shape = face_utils.shape_to_np(shape)
                x1 = int((shape[37][0]+shape[38][0]+shape[41][0]+shape[40][0])/4)
                y1 = int((shape[37][1]+shape[38][1]+shape[41][1]+shape[40][1])/4)
                x2 = int((shape[43][0]+shape[44][0]+shape[46][0]+shape[47][0])/4)
                y2 = int((shape[43][1]+shape[44][1]+shape[46][1]+shape[47][1])/4)
                x3 = shape[30][0]
                y3 = shape[30][1]
                x4 = shape[48][0]
                y4 = shape[48][1]
                x5 = shape[54][0]
                y5 = shape[54][1]
                mrk_data.append([filename,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5])
                dots = [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x5,y5]]
                if (cnt % 100) == 0:
                    print(cnt,'/60000')
                    try:
                        for (x,y) in dots:
                            img = cv2.circle(img,(x,y),2,(0,0,255),-1)
                        cv2.imwrite(os.path.join(save_directory, filename), img)
                    except Exception as e:
                        print(str(e))
    df1 = pd.DataFrame(mrk_data, columns=['filename',
                                         'x1','y1','x2','y2','x3','y3','x4','y4','x5','y5'])
    return df1

In [ ]:
directory = r"D:\Dataset\aihub_korean_emotion\cropped\_____"
save_directory = r"D:\Dataset\aihub_korean_emotion\landmark-samples\_____"
df = pd.read_csv("D:/Dataset/aihub_korean_emotion/csvs/dealt-boundary-problem/dealt-boundary-_____.csv", index_col=0)
df0 = draw_and_save_landmarks_ratio(directory, save_directory, df)
df0.to_csv('landmark______.csv')

출력 - 랜드마크 찍은 이미지 저장경로에 write하고 랜드마크 좌표값 5개x2=10개를 데이터 프레임으로 저장하여 반환

CSV 파일 관련
4단계 이후 나오는 dataframe에서 x_Top, y_Top, x_Bottom, y_Bottom은 원본 이미지를 기준으로 retinaface가 탐지한 얼굴 영역의 boundingbox 값이다.
이후 실제로 잘려지는 부분의 좌표는 (x_Top-margin_left, y_Top-margin_top) ~ (x_Bottom+margin_right,y_Bottom+margin_bottom) 이다.
이 영역이 640x640으로 축소된다.
그 축소된 640by640이미지 기준으로 랜드마크 detection을 수행했기 때문에
단계 6 이후 나오는 dataframe에서 랜드마크 좌표 10개는 cropped image 기준 상이다.